<a href="https://colab.research.google.com/github/ratm123/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG2/Jose_Luis_Llinares_Anton_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG2. Algoritmos de optimizacion

Jose Luis Llinares Anton

https://colab.research.google.com/drive/1mML16AwuFEfR9jVMd7jDaHvcYc1Em-gm


https://github.com/ratm123/03MAIR---Algoritmos-de-Optimizacion---2019


# Programación dinámica
## Problema 1. Viaje por el rio

In [20]:
def precios(tarifas):
    N = len(tarifas[0])
    precios = [[9999]*N for i in [9999]*N]
    ruta = [[""]*N for i in [""]*N]
    
    for i in range(N-1):
        for j in range(i+1, N):
            MIN = tarifas[i][j]
            ruta[i][j] = i
            
            for k in range(i, j):
                if precios[i][k] + tarifas[k][j] < MIN:
                    MIN = min(MIN, precios[i][k] + tarifas[k][j])
                    ruta[i][j] = k
                precios[i][j] = MIN
                
    return precios, ruta

TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

PRECIOS, RUTA = precios(TARIFAS)
print(PRECIOS, RUTA)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]] [['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


In [19]:
def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA,desde,RUTA[desde][hasta])) + ' - ' + str(RUTA[desde][hasta])
  
calcular_ruta(RUTA, 0, 6)

'0 - 0 - 2 - 5'

## Problema 2. Asignación de tareas

In [18]:
#4 agentes, 4 tareas
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

def valor(S, COSTES):
    VALOR = 0
    for i in range(len(S)):
        VALOR += COSTES[S[i]][i]
    return VALOR
    
print(valor((0, 1, 2, 3), COSTES))

import itertools
def fuerza_bruta(COSTES):
    mejor_valor = 10e10
    mejor_solucion = ()
  
    for s in list(itertools.permutations(range(len(COSTES)))):
        valor_temp = valor(s, COSTES)
        if valor_temp < mejor_valor:
            mejor_valor = valor_temp
            mejor_solucion = s
      
    print("La mejor solución es: ", mejor_solucion, "con valor:", mejor_valor)
      
  
  
fuerza_bruta(COSTES)

73
La mejor solución es:  (0, 3, 1, 2) con valor: 61


### Función para estimar una cota inferior para una solución parcial:

In [0]:
def CI(S, COSTES):
    VALOR = 0
    for i in range(len(S)):
        VALOR += COSTES[i][S[i]]
        
    ## estimacion
    for i in range(len(COSTES)):
        if i not in S:
            VALOR += min([COSTES[j][i] for j in range(len(S), len(COSTES))])
    return VALOR

### Función para ramificar:


In [0]:
def crear_hijos(NODO, N):
    HIJOS = []
    for i in range(N):
        if i not in NODO:
            HIJOS.append({'s': NODO+(i,)})
    return HIJOS

### Funcion principal:

In [15]:
def ramificacion_y_poda(COSTES):
    DIMENSION = len(COSTES)  
    MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
    CotaSup = valor(MEJOR_SOLUCION,COSTES)

    NODOS=[]
    NODOS.append({'s':(), 'ci':CI((),COSTES)} )
    iteracion = 0

    while( len(NODOS) > 0):
        iteracion +=1
        nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
        #Ramificacion
        #Se generan los hijos
        HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]
        NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
        if len(NODO_FINAL ) >0: 
          if NODO_FINAL[0]['ci'] < CotaSup:
            CotaSup = NODO_FINAL[0]['ci']
            MEJOR_SOLUCION = NODO_FINAL
        #Poda
        HIJOS=[x for x in HIJOS if x['ci']<CotaSup]

        #Añadimos los hijos 
        NODOS.extend(HIJOS) 

        #Eliminamos el nodo ramificado
        NODOS =  [x for x in NODOS if x['s'] != nodo_prometedor]

    print("La solución final es: ", MEJOR_SOLUCION, " en ", iteracion, " iteraciones", " para dimension: ", DIMENSION)

ramificacion_y_poda(COSTES)

La solución final es:  [{'s': (0, 2, 3, 1), 'ci': 61}]  en  14  iteraciones  para dimension:  4
